In [ ]:
using JuMP
using SCS # please install the package SCS with support for SDP
using LinearAlgebra
using Plots

# Testbed: over a single instance

Before we compare and benchmark over a large number of random instances, you may test the validity of your program on a single instance first. We begin by specifying the problem dimensions as follows:

In [ ]:
n = 10 # m = n
noise_power = 0.2;

Next we generate the random $y,A$ and the ground truth binary vector $x_{true} \in \{-1,1\}^n$. 

In [ ]:
A = randn(n,n) / sqrt(n); # generate the A matrix
x_true = 2*(rand(n) .< 0.5).-1;
y = A*x_true + noise_power*randn(n); # generate the observation

Now, we have $y,A$ and the ground truth $x_{true}$, and we can try to recover the binary vector x_true.

### Brute-force approach:

The following function generates a list of all $m$-element binary vectors.

In [ ]:
m = 3; list_bin = reverse.(Iterators.product(fill([-1,1],m)...))[:]

The elements are given in the tuple format, which can be accessed as an array by

In [ ]:
[i for i in list_bin[1]]

With the above code, write a function to compute the brute-force BLS solution as follows:

In [ ]:
# we better write a function here
function brute_force_bls( y, A )
    n = size(A,1);
    # generate all the possible combinations!
    list_bin = reverse.(Iterators.product(fill([-1,1],n)...))[:]
    # initialize the variable "x_best"
    x_best = [i for i in list_bin[1]];
    # your code here
    # tips: you may run a for loop here to go through every combinations generated in "list_bin" and save the 
    #       one with the smallest objective value, The number of combinations in "list_bin" is given by 2^n.
    #       notice that to compute the norm of a vector "x", you may simply type "norm(x)"
    
    return x_best # it should output the best solution found in the list_bin
end

In [ ]:
x_bf = brute_force_bls( y, A );

We can compute the error relative to x_true.

In [ ]:
no_error_bf = sum(abs.(x_bf - x_true)) / 2

### SDR Approach
- we shall use Julia/JuMP to solve the SDP involved. We first construct the C matrix:

In [ ]:
C = ; # fill in the "C" here

- Next, we set up the JuMP model using SCS as the solver

In [ ]:
model = Model(SCS.Optimizer);

- the variable specification are similar to before. 
- the diagonal constraint should be enforced such that 
$$ X_{i,i} = 1,~i=1,...,n+1 $$
please refer to the notebook for the Bishop problem to see how to specify $n+1$ constraints in one line.
- for $X$ to be PSD, in JuMP, we use the command `@SDconstraint(model, X >= zeros(n+1, n+1));'. For reference, see http://www.juliaopt.org/JuMP.jl/v0.21/constraints/#Semidefinite-constraints-1
- in Julia, we use `Tr(X)' to represent the trace of X. 

In [ ]:
@variable( model, X[1:n+1,1:n+1] );
@SDconstraint(model, X >= zeros(n+1, n+1));
##### fill in the constraint and objective here ###

###################################################
optimize!(model);

- Finally, we round off the solution by taking the signs in the last column of X.
- Moreover, we can compute the no. of errors made relative to x_true.

In [ ]:
x_SDR_sign = sign.(value.(X[1:n,n+1]));
no_error_SDR = sum(abs.( x_SDR_sign - x_true)) / 2
# remark: the round-off procedure here is different from the randomization technique introduced in the lecture

# Actual Simulation over 100 trials across choices of $n$

After verifying that your Ordinary LS, and SDR codes work,  you can input the code into the respective sections below. The *average error* performance will then be computed.

In [ ]:
n_choice = [5,10,15]
noise_power = 0.2; max_trial = 100;

store_obj_bf = [];
store_obj_sdr = [];

for n in n_choice
    obj_value_bf = 0;
    obj_value_sdr = 0;
    for trial = 1:max_trial
        A = randn(n,n) / sqrt(n); # generate the A matrix
        x_true = 2*(rand(n) .< 0.5).-1;
        y = A*x_true + noise_power*randn(n); # generate the observation
        
        ###### This Block uses the Brute-force #########
        x_bf = brute_force_bls( y, A );
        
        # let the binary solution after round-off be x_ls_sign
        obj_value_bf += ; # calculate the obj value wrt x_bf
        
        ###### This Block uses the SDR ################
        C = ; # fill in the C here
        model = Model(SCS.Optimizer);
        @variable( model, X[1:n+1,1:n+1] );
        @SDconstraint(model, X >= zeros(n+1, n+1));
        
        # your code here 
        
        set_silent(model);
        optimize!(model);
        
        x_sdr_sign = sign.(value.(X[1:n,n+1]));
        
        # let the binary solution after round-off be x_sdr_sign
        obj_value_sdr += ; # calculate the obj value wrt x_sdr_sign
    end
    # this compute the average error
    push!(store_obj_bf, obj_value_bf / (max_trial) );
    push!(store_obj_sdr, obj_value_sdr / (max_trial) );
end

In [ ]:
# the following code helps you with visualizing the SDR vs Brute-force comparison
plot( n_choice, [store_obj_bf,store_obj_sdr], xlabel = "n=m", ylabel = "Objective value", 
    label = ["Brute-force" "SDR"], lw = 3, yaxis=:log , title = "Comparing avg obj value of Brute-force and SDR")